# 损失函数

## 匹配策略

+ 第一个原则：从ground truth box出发，寻找与每一个ground truth box有最大的jaccard overlap的prior bbox，这样就能保证每一个groundtruth box一定与一个prior bbox对应起来(jaccard overlap就是IOU，如图3-26所示，前面介绍过)。 反之，若一个prior bbox没有与任何ground truth进行匹配，那么该prior bbox只能与背景匹配，就是负样本。
一个图片中ground truth是非常少的，而prior bbox却很多，如果仅按第一个原则匹配，很多prior bbox会是负样本，正负样本极其不平衡，所以需要第二个原则。

+ 第二个原则：从prior bbox出发，对剩余的还没有配对的prior bbox与任意一个ground truth box尝试配对，只要两者之间的jaccard overlap大于阈值（一般是0.5），那么该prior bbox也与这个ground truth进行匹配。这意味着某个ground truth可能与多个Prior box匹配，这是可以的。但是反过来却不可以，因为一个prior bbox只能匹配一个ground truth，如果多个ground truth与某个prior bbox的 IOU 大于阈值，那么prior bbox只与IOU最大的那个ground truth进行匹配。

注意：第二个原则一定在第一个原则之后进行，仔细考虑一下这种情况，如果某个ground truth所对应最大IOU的prior bbox小于阈值，并且所匹配的prior bbox却与另外一个ground truth的IOU大于阈值，那么该prior bbox应该匹配谁，答案应该是前者，首先要确保每个ground truth一定有一个prior bbox与之匹配。

```pytorch
overlap_for_each_prior,object_for_each_prior=overlap.max(dim=0)
_,prior_for_each_object=overlap.max(dim=1)

object_for_each_prior[prior_for_each_object]=torch.LongTensor(range(n_objects)).to(device)
overlap_for_each_prior[prior_for_each_object]=1

label_for_each_prior=labels[i][object_for_each_prior]
label_for_each_prior[overlap_for_each_prior<threshhold]=0
```

### 解析
第一句得到一个1x441大小的overlap tensor和一个1x441大小的index tensor

分别表示得到每一个先验框的最大重叠和与之对应的物体索引

第二句得到一个1xn_objects大小的index tensor

这里并没有计算overlap的原因是，根据原则一在二之前，无论overlap为多少都会被计算在内

第三句挨个给物体匹配到的先验框赋值物体的标号，因为是得到的prior_for_each_object是顺序的，所以使用range()，就像```[a,b,c]=[1,2,3]```

所以第四句就赋值就将第二局的index对应的overlap赋值为1

第五句就接着将每一个prior对应的物体全部给label

第六句将不符合条件的prior的label置为0

## 损失函数

敲公式太累，就不写了

## hard negative mining
值得注意的是，一般情况下negative prior bboxes数量 >> positive prior bboxes数量，直接训练会导致网络过于重视负样本，预测效果很差。为了保证正负样本尽量平衡，我们这里使用SSD使用的在线难例挖掘策略(hard negative mining)，即依据confidience loss对属于负样本的prior bbox进行排序，只挑选其中confidience loss高的bbox进行训练，将正负样本的比例控制在positive：negative=1:3。其核心作用就是只选择负样本中容易被分错类的困难负样本来进行网络训练，来保证正负样本的平衡和训练的有效性。